<a href="https://colab.research.google.com/github/atherfawaz/BERT-Supervised/blob/master/RoBERTa_Supervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/drive/My Drive/Ebryx/blogs_train

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
[Errno 2] No such file or directory: '/content/drive/My Drive/Ebryx/blogs_train'
/content


In [2]:
# Code for TPU packages install
!curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5115  100  5115    0     0  26640      0 --:--:-- --:--:-- --:--:-- 26640
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200515 ...
     |████████████████████████████████| 61kB 2.8MB/s 
Uninstalling torch-1.6.0+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.6.0+cu101
Uninstalling torchvision-0.7.0+cu101:
  Successfully uninstalled torchvision-0.7.0+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200515-cp36-cp36m-linux_x86_64.whl...
| [1 files][ 91.0 MiB/ 91.0 MiB]                                                
Operation completed over 1 objects/91.0 MiB.                                     


In [1]:
!pip install -q transformers

# Importing stock libraries
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importing lackages from our NLP-Hugging Package
from transformers import RobertaConfig, RobertaModel, RobertaTokenizerFast, RobertaForSequenceClassification

In [2]:
# TPU
import torch_xla
import torch_xla.core.xla_model as xm
from sklearn.utils import shuffle
device = xm.xla_device()

In [4]:
%cd /content/gdrive/My Drive/Ebryx/blogs_train
# Creating the dataset and dataloader for the neural network
df = pd.read_csv('/content/gdrive/My Drive/Ebryx/dataset.csv')
df = df[['Post', 'Sign', 'Gender']]

encode_dict = {}

def encode_cat(x):
    if x not in encode_dict.keys():
      encode_dict[x]=len(encode_dict)
    return encode_dict[x]

df['Sign'] = df['Sign'].apply(lambda x: encode_cat(x))
df['Post'] = df['Post'].str.replace('\r','').str.replace('\t','').str.replace('\xa0', '')

df = df [['Post', 'Sign']]

df = shuffle(df)
df.head(10)

/content/gdrive/My Drive/Ebryx/blogs_train


,Post,Sign
72872,"""The difference between us and nerds is...",11
20097,nothing much to say today... its just...,5
354659,Today was sort of fun. I had a re...,0
345094,Major prayer request: This is still jus...,2
68858,grr. Ive been in a bad mood for a few d...,4
29585,Jason Kan is a nice guy...seriously. He...,0
82489,urlLink I like this contemplative h...,8
76804,"Oh, I wouldn't consider myself all ...",7
56050,Look Kevin I'm really tired of yo...,7
325670,The story of God ordering Abraham to sa...,4


In [5]:
# Creating a CustomDataset class that is used to read the updated dataframe and tokenize the text. 
# The class is used in the return_dataloader function

class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        post = str(self.data.Post[index])
        post = " ".join(post.split())
        inputs = self.tokenizer.encode_plus(
            post,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.Sign[index], dtype=torch.float)
        } 
    
    def __len__(self):
        return self.len


# Creating the dataset and dataloader for the neural network

MAX_LEN = 512
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=200)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
testing_set = Triage(test_dataset, tokenizer, MAX_LEN)

train_params = {'batch_size': TRAIN_BATCH_SIZE,
            'shuffle': True,
            'num_workers': 1
            }

val_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 1
                }

training_loader = DataLoader(training_set, **train_params)
validation_loader = DataLoader(testing_set, **val_params)

FULL Dataset: (380720, 2)
TRAIN Dataset: (304576, 2)
TEST Dataset: (76144, 2)


In [6]:
# Creating the customized model, by adding a drop out and a dense layer on top of roberta to get the final output for the model. 

class ModelClass(torch.nn.Module):
    def __init__(self):
        super(ModelClass, self).__init__()
        self.model_layer = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 12)

    def forward(self, input_ids, attention_mask):
        output_1 = self.model_layer(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

# Function to return model based on the defination of Model Class
def return_model(device):
    model = ModelClass()
    model = model.to(device)
    return model


# Function to calcuate the accuracy of the model
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [ ]:
model = ModelClass()
model.to(device)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):
  tr_loss = 0
  n_correct = 0
  nb_tr_steps = 0
  nb_tr_examples = 0
  model.train()
  for i,data in enumerate(training_loader, 0):
      ids = data['ids'].to(device, dtype = torch.long)
      mask = data['mask'].to(device, dtype = torch.long)
      targets = data['targets'].to(device, dtype = torch.long)

      outputs = model(ids, mask)
      loss = loss_function(outputs, targets)
      tr_loss += loss.item()
      big_val, big_idx = torch.max(outputs.data, dim=1)
      n_correct += calcuate_accu(big_idx, targets)

      nb_tr_steps += 1
      nb_tr_examples+=targets.size(0)
      
      if i%5==0:
          loss_step = tr_loss/nb_tr_steps
          accu_step = (n_correct*100)/nb_tr_examples 
          print(f"[ {i} ] Loss: {loss_step:.3f} | Accuracy: {accu_step:.3f}")

      optimizer.zero_grad()
      loss.backward()
      # # When using GPU
      #optimizer.step()

       # When using TPU
      xm.optimizer_step(optimizer)
      xm.mark_step()

  print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
  epoch_loss = tr_loss/nb_tr_steps
  epoch_accu = (n_correct*100)/nb_tr_examples
  print(f"Training Loss Epoch: {epoch_loss}")
  print(f"Training Accuracy Epoch: {epoch_accu}")

[ 0 ] Loss: 2.523 | Accuracy: 6.250
[ 5 ] Loss: 2.513 | Accuracy: 7.292
[ 10 ] Loss: 2.498 | Accuracy: 6.250
[ 15 ] Loss: 2.505 | Accuracy: 6.250
[ 20 ] Loss: 2.503 | Accuracy: 6.250
[ 25 ] Loss: 2.497 | Accuracy: 7.212
[ 30 ] Loss: 2.499 | Accuracy: 6.855
[ 35 ] Loss: 2.500 | Accuracy: 6.944
[ 40 ] Loss: 2.496 | Accuracy: 8.079
[ 45 ] Loss: 2.492 | Accuracy: 8.288
[ 50 ] Loss: 2.491 | Accuracy: 8.333
[ 55 ] Loss: 2.491 | Accuracy: 8.371
[ 60 ] Loss: 2.490 | Accuracy: 8.299
[ 65 ] Loss: 2.489 | Accuracy: 8.523
[ 70 ] Loss: 2.488 | Accuracy: 8.627
[ 75 ] Loss: 2.488 | Accuracy: 8.635
[ 80 ] Loss: 2.488 | Accuracy: 8.719
[ 85 ] Loss: 2.487 | Accuracy: 8.576
[ 90 ] Loss: 2.488 | Accuracy: 8.791
[ 95 ] Loss: 2.487 | Accuracy: 8.724
[ 100 ] Loss: 2.487 | Accuracy: 8.787
[ 105 ] Loss: 2.486 | Accuracy: 8.903
[ 110 ] Loss: 2.487 | Accuracy: 8.840
[ 115 ] Loss: 2.487 | Accuracy: 8.890
[ 120 ] Loss: 2.487 | Accuracy: 9.091
[ 125 ] Loss: 2.486 | Accuracy: 9.028
[ 130 ] Loss: 2.486 | Accuracy: 9.